# Markov Decision Process

## Section 1: Introduction

In this section, we will introduce the concepts of MDP, Q-values, and V-values. These concepts are fundamental to the field of AI and machine learning, as they are used to model **decision-making problems** in various domains such as "robotics", "finance", and "healthcare".

MDP stands for Markov Decision Process. It is a mathematical framework for modeling decision-making problems in which the outcomes are partly random and partly under the control of a decision-maker. MDPs are defined by a set of states, a set of actions, a reward function, and a transition function. The goal is to find a policy that maximizes the expected cumulative reward over time.

Q-values and V-values are two important concepts in the context of MDPs. A Q-value represents the expected cumulative reward of taking a particular action in a particular state and following a specific policy thereafter. A V-value represents the expected cumulative reward of being in a particular state and following a specific policy thereafter. These values are used to evaluate and improve the policy of an agent in an MDP.

## Section 2: The Basics of MDPs

In this section, we will explain the basic components of an MDP.

An MDP is defined by "a set of states", "a set of actions", "a reward function", and "a transition function". The state space is the set of all possible states that the agent can be in. The action space is the set of all possible actions that the agent can take. The reward function defines the reward the agent receives for each action taken in a particular state. The transition function defines the probability of moving from one state to another state after taking a particular action.

To illustrate these concepts, let's consider an example of a **robot that needs to navigate through a maze**. The robot can be in one of several states, such as at the start of the maze, at a junction in the maze, or at the end of the maze. This robot takes an action. With Probability of **0.8** It goes in that desired direction but with probability of **0.2** It goes in the perpendicular direction (0.1, 0.1 for each)!

In an MDP, the agent interacts with the environment by selecting actions based on its current state and the expected future reward. The goal of the agent is to find a policy that maximizes the expected cumulative reward over time.

**QUESTION**

1. What are the state space, action space, reward function, and transition function of the robot in the maze example? Explain why you think each of these components is important for the robot to navigate through the maze.

The state space is the coordinates of robot. The actions are going to a direction which is probably left right up down. Reward function can be defined by ourselves. For example the difference of manhattan distance between robot and exit. Transition function is the probability of going to some other state from a state by taking a specific action.

With all of these components, we can simulate the environment and simulate or robot.

2. Is our environment stochastic or deterministic? Why?! Stochastic; Because we don't know the outcome of our actions.

**Define The MDP**:

Based on your choice of rewards and transitions and the state space, define the MDP for the robot in the maze example. You can complete the following code to define the MDP:

In [1]:
from typing import Literal
import numpy as np
from enum import Enum

# Definition of the maze
maze = np.array([[2, 0, 0, 0, 0],
                 [0, 1, 1, 0, 1],
                 [0, 0, 0, 0, 0],
                 [0, 1, 1, 1, 3]])

print("Our Maze is: \n", maze)

# Define the states and actions
class Action(Enum):
    LEFT = (0,-1)
    RIGHT = (0,1)
    UP = (-1,0)
    DOWN = (1,0)
    @staticmethod
    def next_state(state: tuple[int, int], action) -> tuple[int, int]:
        return (state[0] + action.value[0], state[1] + action.value[1])
        
    def perpendicular(self):
        if self == Action.LEFT or self == Action.RIGHT:
            return [Action.UP, Action.DOWN]
        return [Action.LEFT, Action.RIGHT]

    def __str__(self) -> str:
        if self == Action.LEFT:
            return "←"
        if self == Action.RIGHT:
            return "→"
        if self == Action.UP:
            return "↑"
        return "↓"
        

# Define the reward function
def state_reward(state: tuple[int, int]) -> int:
    #return (maze.shape[0] + maze.shape[1]) - (abs(maze.shape[0] - 1 - state[0]) + abs(maze.shape[1] - 1 - state[1]))
    if state[0] == maze.shape[0] - 1 and state[1] == maze.shape[1] - 1:
        return 1
    return 0

# Define the transition probabilities
def valid_next_state(state: tuple[int, int], action: Action) -> bool:
    next_state = Action.next_state(state, action)
    if next_state[0] < 0 or next_state[1] < 0:
        return False
    if next_state[0] >= maze.shape[0] or next_state[1] >= maze.shape[1]:
        return False
    if maze[next_state[0], next_state[1]] == 1:
        return False
    return True

def transition_probs(state1: tuple[int, int], action: Action, state2: tuple[int, int]) -> float:
    next_predicted_state = Action.next_state(state1, action)
    if not valid_next_state(state1, action) and next_predicted_state == state2:
        return 0
    # Check other states which we can go in
    valid_other_states: list[tuple[int, int]] = []
    for a in action.perpendicular():
        if valid_next_state(state1, a):
            valid_other_states.append(Action.next_state(state1, a))
    # Normal move
    if next_predicted_state == state2:
        return 0.8
    # Perpendicular move
    if state2 in valid_other_states:
        return 0.1
    # Something else
    return 0

# Set the discount factor (for further use in v-value iteration and q-value iteration)
discount = 0.9

# Define the initial value function (you can simply set all to 0)
values = np.zeros(maze.shape)
values[-1, -1] = 1

# Define the initial Q function (you can simply set all to 0)
q_values = np.zeros(maze.shape + (len(Action),))


Our Maze is: 
 [[2 0 0 0 0]
 [0 1 1 0 1]
 [0 0 0 0 0]
 [0 1 1 1 3]]


## Section 3: Computing V-values

In this section, we will explain how to compute V-values for an MDP using the Bellman equation.

The Bellman equation is a recursive equation that expresses the value of a state in terms of the values of its successor states. It is defined as:

$$V(s) = R(s) + \gamma * \max_a (\sum_{s'} P(s, a, s') * V(s'))$$

where V(s) is the value of state s, R(s) is the reward for being in state s, γ is the discount factor that determines the importance of future rewards, P(s, a, s') is the probability of moving from state s to state s' after taking action a, and max_a is the maximum over all possible actions a.

To compute the V-values for an MDP, we start with an initial estimate of the V-values and update them iteratively using the Bellman equation until they converge to the true values. The update rule is:

$$V(s) \leftarrow R(s) + \gamma * \max_a (\sum_{s'} P(s, a, s') * V(s'))$$

We can use dynamic programming algorithms such as value iteration or policy iteration to compute the V-values.

We can use the Bellman equation to compute the V-values for each state in the maze. The V-values represent the expected cumulative reward that the robot can obtain if it starts from that state and follows an optimal policy thereafter. Complete the code below:

(**Note:** your final result can be slightly different from the result printed below and it's okay!)

In [27]:
# Initiate again
values = np.zeros(maze.shape)
values[-1, -1] = 1
# Do the thing
ITERATIONS = 100
for _ in range(ITERATIONS):
    for (x, y), _ in np.ndenumerate(values):
        if maze[x, y] == 1: # void
            continue
        best_action = 0 # The arg max
        for a in Action:
            valid_next_states: list[tuple[int, int]] = [] # States which are valid from this state
            for a2 in Action:
                if valid_next_state((x, y), a2):
                    next_state = Action.next_state((x, y), a2)
                    valid_next_states.append(next_state)
            s = 0 # sum
            for next_state in valid_next_states:
                s += transition_probs((x, y), a, next_state) * values[next_state[0], next_state[1]]
            best_action = max(best_action, s)
        values[x, y] = state_reward((x, y)) + discount * best_action

# Print the V-values
print(values)


[[0.33622712 0.41509521 0.57652112 0.80072378 0.57652112]
 [0.41509521 0.         0.         0.96798608 0.        ]
 [0.57652112 0.69694998 0.96798608 1.34442512 1.74625885]
 [0.41509521 0.         0.         0.         2.25730637]]


## Section 4: Computing Q-values

In this section, we will explain how to compute Q-values for an MDP using the Bellman equation.

The Q-values represent the expected cumulative reward that the robot can obtain if it starts from a particular state and takes a particular action, and then follows an optimal policy thereafter. The Q-values can be computed using the Bellman equation as follows:

$$Q(s, a) = \sum_{s'} P(s, a, s') (R(s, a) + \gamma \max_{a'} (Q(s', a')))$$

where Q(s, a) is the Q-value of state-action pair (s, a), R(s, a) is the reward for taking action a in state s, γ is the discount factor that determines the importance of future rewards, P(s, a, s') is the probability of moving from state s to state s' after taking action a, max_a' is the maximum over all possible actions a' in state s', and sum_s' is the sum over all possible successor states s' of state s.

To compute the Q-values for an MDP, we start with an initial estimate of the Q-values and update them iteratively using the Bellman equation until they converge to the true values. The update rule is:

$$Q(s, a) = \sum_{s'} P(s, a, s') (R(s, a) + \gamma \max_{a'} (Q(s', a')))$$

We can use dynamic programming algorithms such as Q-learning or SARSA to compute the Q-values.


We can use the Q-learning algorithm to compute the Q-values for each state-action pair in the maze. The Q-values represent the expected cumulative reward that the robot can obtain if it starts from a particular state and takes a particular action, and then follows an optimal policy thereafter. Complete the code below:

(**Note:** your final result can be slightly different from the result printed below and it's okay!)

In [2]:
# Re-initiate
q_values = np.zeros(maze.shape + (len(Action),))
# Compute Q-Values using Bellman equations
ITERATIONS = 100
for _ in range(ITERATIONS):
    for (x, y, a), _ in np.ndenumerate(q_values):
        if maze[x, y] == 1: # void
            continue
        s = 0 # Sum
        valid_next_states: list[tuple[int, int]] = []
        for action in Action:
            if valid_next_state((x, y), action):
                next_state = Action.next_state((x, y), action)
                valid_next_states.append(next_state)
        for next_state in valid_next_states:
            action = list(Action)[a]
            best_q = 0
            for a2 in Action:
                best_q = max(best_q, q_values[next_state[0], next_state[1], list(Action).index(a2)])
            s += transition_probs((x, y), action, next_state) * (state_reward((x, y)) + discount * best_q)
        q_values[x, y, a] =  s

# Print the Q-values
print(q_values)


[[[0.02988686 0.2689817  0.02988686 0.2689817 ]
  [0.19366682 0.33207617 0.06571787 0.06571787]
  [0.23909484 0.4612169  0.08753897 0.08753897]
  [0.40177117 0.40177117 0.08301904 0.64057903]
  [0.4612169  0.         0.05765211 0.05765211]]

 [[0.06571787 0.06571787 0.19366682 0.33207617]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.15445072 0.15445072 0.4612169  0.77438887]
  [0.         0.         0.         0.        ]]

 [[0.05977371 0.4612169  0.28927524 0.28927524]
  [0.33207617 0.55755998 0.11120452 0.11120452]
  [0.40144319 0.77438887 0.14697901 0.14697901]
  [0.62725498 1.07554009 0.75298562 0.19542564]
  [0.93691493 0.16252606 0.09679861 1.39700708]]

 [[0.04150952 0.04150952 0.33207617 0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.22573064 0.22573064 1.8058451  0.        ]]]


## Section 5: Visualizing the Optimal Policy

Now that we have computed the Q-values, we can use them to find the optimal policy, which is the sequence of actions that the robot should take in each state to maximize its expected reward. We can visualize the optimal policy as arrows in a grid, where each arrow corresponds to the action with the highest Q-value in the corresponding state. Complete the code below:

(**Note:** your final result can be slightly different from the result printed below and it's okay!)

In [3]:
# Compute the optimal policy

policy: list[list[str]] = []
for i in range(maze.shape[0]):
    policy_row: list[str] = []
    for j in range(maze.shape[1]):
        if maze[i, j] == 1:
            policy_row.append("×")
            continue
        q_values_of_state = list(q_values[i, j])
        action = list(Action)[q_values_of_state.index(max(q_values_of_state))]
        policy_row.append(str(action))
    policy.append(policy_row)

print(policy)

[['→', '→', '→', '↓', '←'], ['↓', '×', '×', '↓', '×'], ['→', '→', '→', '→', '↓'], ['↑', '×', '×', '×', '↑']]


In [5]:
for row in policy:
    print(' '.join(row))

print()
print()

for row in maze:
    for cell in row:
        if cell == 0:
            print("P", end="")
        if cell == 1:
            print("X", end="")
        if cell == 2:
            print("S", end="")
        if cell == 3:
            print("G", end="")
        print(" ", end="")
    print()

→ → → ↓ ←
↓ × × ↓ ×
→ → → → ↓
↑ × × × ↑


S P P P P 
P X X P X 
P P P P P 
P X X X G 
